# E09 Movielens 영화 추천

-  MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작
- MovieLens 1M Dataset
- 별점 데이터를 Implicit 데이터로 간주하고 테스트 ( 별점을 시청횟수로 해석해서 생각)
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외



## 1. 데이터 준비와 전처리

In [1]:
import pandas as pd
PATH = '/content/drive/MyDrive/Github/AIFFEL/E09'

In [2]:
rating_file_path= PATH + '/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(3)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [3]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꾸기
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings.head(3)

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


In [6]:
# 영화 제목을 보기 위해 메타 데이터 읽어오기
movie_file_path= PATH + '/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
#movie_id로 inner join
data = pd.merge(movies, ratings, left_on='movie_id', right_on='movie_id', how='inner')
data.head()

,movie_id,title,genre,user_id,count,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [8]:
data.isnull().sum()

movie_id     0
title        0
genre        0
user_id      0
count        0
timestamp    0
dtype: int64

In [9]:
# # # 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'title', 'count']
data = data[using_cols]
data.head(10)

,user_id,title,count
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [10]:
#검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔 줍시다.
data['title'] = data['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,title,count
0,1,toy story (1995),5
1,6,toy story (1995),4
2,8,toy story (1995),4
3,9,toy story (1995),5
4,10,toy story (1995),5
5,18,toy story (1995),4
6,19,toy story (1995),5
7,21,toy story (1995),3
8,23,toy story (1995),4
9,26,toy story (1995),3


In [11]:
#첫번째 유저가 어떤 영화를 봤는지
condition = (data['user_id']== data.loc[0, 'user_id'])
data.loc[condition]

,user_id,title,count
0,1,toy story (1995),5
19773,1,pocahontas (1995),5
35123,1,apollo 13 (1995),5
54895,1,star wars: episode iv - a new hope (1977),4
116019,1,schindler's list (1993),5
119008,1,"secret garden, the (1993)",4
130190,1,aladdin (1992),4
138999,1,snow white and the seven dwarfs (1937),4
139706,1,beauty and the beast (1991),5
142605,1,fargo (1996),4


## 2. 분석하기
ratings에 있는 유니크한 영화 개수


In [12]:
data['title'].nunique()

3628

rating에 있는 유니크한 사용자 수

In [13]:
data['user_id'].nunique()

6039

가장 인기 있는 영화 30개(인기순)

In [14]:
movie_count = data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

유저별 평가한 영화 수 통계

In [15]:
user_count = data.groupby('user_id')['count'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: count, dtype: float64

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [16]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Jumanji (1995)' ,'Star Wars: Episode IV - A New Hope (1977)' ,'Matrix, The (1999)','terminator 2: judgment day (1991)','Toy Story (1995)']

# 'jungin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
# my_movielist = pd.DataFrame({'user_id': ['jungin']*5, 'movie_id': data[data["title"].isin(my_favorite)]["movie_id"], 'title': my_favorite, 'count': [5]*5})
my_movielist = pd.DataFrame({'user_id': ['jungin']*5, 'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':['jungin']})['user_id'].any():  # user_id에 'jungin'이라는 데이터가 없다면
    data = data.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,count
836473,5682,"contender, the (2000)",3
836474,5812,"contender, the (2000)",4
836475,5831,"contender, the (2000)",3
836476,5837,"contender, the (2000)",4
836477,5998,"contender, the (2000)",4
0,jungin,Jumanji (1995),5
1,jungin,Star Wars: Episode IV - A New Hope (1977),5
2,jungin,"Matrix, The (1999)",5
3,jungin,terminator 2: judgment day (1991),5
4,jungin,Toy Story (1995),5


## 4. CSR matrix 만들기

In [17]:
#user_id와 title에 인덱싱을 하는 작업
# 고유한 유저, title를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [18]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['jungin']) # 6039-1 ????
print(movie_to_idx['Toy Story (1995)']) #3628+5(전체영화수)-1(인덱스는0부터) = 3632

6039
3631


In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수 : dic.get('key') = value
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,count
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,3628,5
1,6039,3629,5
2,6039,3630,5
3,6039,569,5


- data의 user_id와 title 값을 인덱스 값으로 변경 완료

In [20]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data["count"], (data["user_id"], data["title"])), shape= (num_user, num_title))
csr_data

<6040x3632 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시키기

In [21]:
# !pip install implicit

- implicit 패키지는 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지이다.
- implicit 패키지의 als(AlternatingLeastSquares) 모델을 사용

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [23]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

- AlternatingLeastSquares 클래스의 __init__ 파라미터

  1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
  2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
  3. use_gpu : GPU를 사용할 것인지 
  4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3632x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6.내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


- jungin 벡터와 toystory의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [26]:
jungin, toystory = user_to_idx['jungin'], movie_to_idx['Toy Story (1995)']
jungin_vector, toystory_vector = als_model.user_factors[jungin], als_model.item_factors[toystory]

In [27]:
# jungin과 toystory를 내적하는 코드
print("toystory 선호도:",np.dot(jungin_vector, toystory_vector))

toystory 선호도: 0.0022418036


In [29]:
#랜덤 영화에 대한 선호도를 어떻게 예측할지?
Matrix = movie_to_idx['Matrix, The (1999)']
Matrix_vector = als_model.item_factors[Matrix]
print("Matrix 선호도:", np.dot(jungin_vector, Matrix_vector))

Matrix 선호도: 0.0022052608


## 7.내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.



In [30]:
favorite_movie = 'Toy Story (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3631, 1.0),
 (3628, 0.99243516),
 (3629, 0.9644737),
 (3630, 0.9512295),
 (95, 0.9190582),
 (586, 0.9161415),
 (1363, 0.91412354),
 (1992, 0.9138639),
 (2566, 0.9132511),
 (710, 0.91310036),
 (564, 0.9122309),
 (1395, 0.9120507),
 (657, 0.91201377),
 (131, 0.911849),
 (2203, 0.91133213)]

In [31]:
#movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Toy Story (1995)',
 'Jumanji (1995)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Matrix, The (1999)',
 'shopping (1994)',
 'century (1993)',
 "brother's kiss, a (1997)",
 'number seventeen (1932)',
 'male and female (1919)',
 'last of the high kings, the (a.k.a. summer fling) (1996)',
 "i don't want to talk about it (de eso no se habla) (1993)",
 'war at home, the (1996)',
 'daens (1992)',
 'neon bible, the (1995)',
 '24 7: twenty four seven (1997)']

In [32]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [34]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Jumanji (1995)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Matrix, The (1999)',
 'shopping (1994)',
 'century (1993)',
 "brother's kiss, a (1997)",
 'number seventeen (1932)',
 'male and female (1919)',
 'last of the high kings, the (a.k.a. summer fling) (1996)']

In [33]:
get_similar_movie('Matrix, The (1999)')

['Matrix, The (1999)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Jumanji (1995)',
 'Toy Story (1995)',
 '24 7: twenty four seven (1997)',
 'shopping (1994)',
 'man from down under, the (1943)',
 'century (1993)',
 "brother's kiss, a (1997)",
 'number seventeen (1932)']

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [37]:
user = user_to_idx['jungin']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(462, 0.28453457),
 (2325, 0.28334603),
 (2657, 0.23385084),
 (106, 0.22787502),
 (1419, 0.22212048),
 (1122, 0.20931733),
 (439, 0.16869749),
 (1098, 0.13974941),
 (1084, 0.1313235),
 (1448, 0.11899364),
 (1391, 0.1120225),
 (1131, 0.10352549),
 (249, 0.101196244),
 (363, 0.100211315),
 (1094, 0.09680673),
 (948, 0.09258194),
 (1415, 0.08997411),
 (2053, 0.08918302),
 (1810, 0.08878404),
 (2689, 0.08837645)]

In [38]:
[idx_to_movie[i[0]] for i in movie_recommended]

['jurassic park (1993)',
 'matrix, the (1999)',
 'total recall (1990)',
 'braveheart (1995)',
 'men in black (1997)',
 'terminator, the (1984)',
 'fugitive, the (1993)',
 'alien (1979)',
 'aliens (1986)',
 'hunt for red october, the (1990)',
 'lost world: jurassic park, the (1997)',
 'nikita (la femme nikita) (1990)',
 'star wars: episode iv - a new hope (1977)',
 'speed (1994)',
 'star wars: episode vi - return of the jedi (1983)',
 'sound of music, the (1965)',
 'face/off (1997)',
 'edward scissorhands (1990)',
 'saving private ryan (1998)',
 'from russia with love (1963)']

In [43]:
terminator = movie_to_idx['terminator, the (1984)']
explain = als_model.explain(user, csr_data, itemid=terminator)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('terminator 2: judgment day (1991)', 0.20297319862374014),
 ('Matrix, The (1999)', 0.0009003418901878228),
 ('Jumanji (1995)', 0.000847419986519316),
 ('Star Wars: Episode IV - A New Hope (1977)', 0.0008443025209238643),
 ('Toy Story (1995)', 0.0008235424194420156)]

In [42]:
ToyStory = movie_to_idx['Toy Story (1995)']
explain = als_model.explain(user, csr_data, itemid=ToyStory)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('terminator 2: judgment day (1991)', 0.002043761117385211),
 ('Toy Story (1995)', 4.075397244375052e-05),
 ('Jumanji (1995)', 3.937179253631186e-05),
 ('Star Wars: Episode IV - A New Hope (1977)', 2.9261792628821034e-05),
 ('Matrix, The (1999)', 2.673113881503534e-05)]

- terminator와 toystory 를 기반으로 내가 좋아할만한 영화를 추천 받았을 때, 각 시리즈물이 가장 높은 점수로 추천되는 것을 확인

## 9. 회고
- implicit 패키지에서 가져 온 als_model에 CSR(Compressed Sparse Row) 을 넣어 학습시키고, als_model.user_factors, als_model.item_factors 를 통해 특정 유저와 아이템의 벡터를 추출, 각 벡터의 유사도를 측정해서 비슷한 아이템인지 확인, 그리고 als_model.recommend 통해 좋아할만한 아이템을 추천해주는 실습을 진행했다.
- 비록 이번에는 explicit data(유저가 자신의 선호도를 직접 표현)인 Rating data를 implicit 패키지로 분석했지만 다음번엔 implicit data로 분석해보고 싶다는 생각이 들었다.
- MF(Matrix Factorization)과 CSR(Compressed Sparse Row) 개념들을 보면서 추천시스템에서 경량화를 위해 어떤 방식을 취하는지 알 수 있었다.